<a href="https://colab.research.google.com/github/Tiabet/Complete_story/blob/master/KoGPT_Finetuning_without_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!pip install accelerate
!pip install datasets

In [2]:
!nvidia-smi

Sat Sep 30 05:44:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, DataCollatorWithPadding,TrainingArguments, Trainer, GPT2Config, RandomSampler, SequentialSampler
from datasets import Dataset, load_dataset
import os
import numpy as np
import random

모델 호출

In [4]:
model_checkpoint = "skt/kogpt2-base-v2"

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_checkpoint, bos_token='<bos>', eos_token='<eos>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', sep_token = '<sep>', padding_side='left')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Load the existing model checkpoint
model_config = GPT2Config.from_pretrained(model_checkpoint)

# Add the new special token to the model's config
model_config.bos_token_id = tokenizer.bos_token_id
model_config.eos_token_id = tokenizer.eos_token_id
model_config.unk_token_id = tokenizer.unk_token_id
model_config.pad_token_id = tokenizer.pad_token_id
model_config.mask_token_id = tokenizer.mask_token_id
model_config.sep_token_id = tokenizer.sep_token_id  # Add this line

# Initialize the model with the updated configuration
model = GPT2LMHeadModel.from_pretrained(model_checkpoint, config=model_config)

model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 51203. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(51203, 768)

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(4)

데이터셋 구축

In [7]:
dataset = load_dataset('drive/MyDrive', data_files={
    'train': 'nikluge-sc-2023-train.jsonl',
    'validation': 'nikluge-sc-2023-dev.jsonl',
    'test': 'nikluge-sc-2023-test.jsonl'
})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
sentence1 = []
sentence3 = []

for dictionary in dataset['train']['input']:
    sentence1.append(f"첫 번째 문장 : {dictionary['sentence1']}")
    sentence3.append(f"세 번째 문장 : {dictionary['sentence3']}")

dataset['train'] = dataset['train'].add_column('sentence1', sentence1)
dataset['train'] = dataset['train'].add_column('sentence3', sentence3)

In [9]:
sentence1 = []
sentence3 = []

for dictionary in dataset['validation']['input']:
    sentence1.append(f"첫 번째 문장 : {dictionary['sentence1']}")
    sentence3.append(f"세 번째 문장 : {dictionary['sentence3']}")

dataset['validation'] = dataset['validation'].add_column('sentence1', sentence1)
dataset['validation'] = dataset['validation'].add_column('sentence3', sentence3)

In [10]:
sentence1 = []
sentence3 = []

for dictionary in dataset['test']['input']:
    sentence1.append(f"첫 번째 문장 : {dictionary['sentence1']}")
    sentence3.append(f"세 번째 문장 : {dictionary['sentence3']}")

dataset['test'] = dataset['test'].add_column('sentence1', sentence1)
dataset['test'] = dataset['test'].add_column('sentence3', sentence3)

In [11]:
def tokenize_function(batch):
    sentences1 = batch['sentence1']
    sentences3 = batch['sentence3']
    outputs = batch['output']

    model_inputs = []

    for s1, s3, output in zip(sentences1, sentences3, outputs):
        # Convert list elements to strings
        sentence1_str = ''.join(map(str, s1))
        sentence3_str = ''.join(map(str, s3))
        output_str = ''.join(map(str,output))

        # Tokenize the input sentences
        inputs = tokenizer(
            '<bos>' + sentence1_str + '<sep>' + sentence3_str +\
            '<sep>' + '두 번째 문장 :'+ output_str + '<eos>',
            padding="max_length",
            max_length=64,  # Adjust this as needed
            truncation=True,
            return_attention_mask = True,
            return_tensors = 'pt'
        )

        # Append tokenized inputs and labels to the lists
        model_inputs.append(inputs)

    return {
        'input_ids': [item['input_ids'] for item in model_inputs],
        'attention_mask': [item['attention_mask'] for item in model_inputs],
        'labels': [item['input_ids'] for item in model_inputs],
    }

In [12]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/120140 [00:00<?, ? examples/s]

Map:   0%|          | 0/15017 [00:00<?, ? examples/s]

Map:   0%|          | 0/15018 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets = tokenized_datasets.remove_columns(dataset["train"].column_names)

In [14]:
print(tokenizer.decode(tokenized_datasets['train']['input_ids'][0][0], skip_special_tokens=False))

<pad><pad><pad><pad><pad><pad><pad><bos> 첫 번째 문장 : 시은이는 다음 주의 여름 휴가 이전에 기분을 전환하고 싶었다.<sep> 세 번째 문장 : 예쁘게 꾸민 손톱을 보며 여행을 갈 생각에 한층 더 들떴다.<sep> 두 번째 문장 :그래서 네일샵에 가서 예쁘게 손톱을 칠했다.<eos>


In [15]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #동적 패딩, attention_mask, input_ids 등의 길이를 똑같이 맞춰줌

훈련

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
batch_size = 64
num_train_epochs = 3
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

train_sampler = RandomSampler(tokenized_datasets["train"])
eval_sampler = SequentialSampler(tokenized_datasets["validation"])

training_args  = TrainingArguments(
    output_dir=f"{model_name}-finetuned-koGPT-complete_story",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=num_train_epochs,
    logging_steps = logging_steps,
    push_to_hub=True,
)

In [22]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    train_sampler=train_sampler,  # Set the train sampler for shuffling
    eval_sampler=eval_sampler,
)


In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.896500,3.177052
2,3.545100,3.165875
3,3.380500,3.054913


TrainOutput(global_step=5634, training_loss=3.2740431837380504, metrics={'train_runtime': 6779.458, 'train_samples_per_second': 53.164, 'train_steps_per_second': 0.831, 'total_flos': 1.177185927168e+16, 'train_loss': 3.2740431837380504, 'epoch': 3.0})

In [24]:
trainer.evaluate()

{'eval_loss': 3.054912805557251,
 'eval_runtime': 83.1662,
 'eval_samples_per_second': 180.566,
 'eval_steps_per_second': 2.826,
 'epoch': 3.0}

In [25]:
trainer.push_to_hub(commit_message="Training complete", tags="text generation")

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

'https://huggingface.co/Tiabet/kogpt2-base-v2-finetuned-koGPT-complete_story/tree/main/'

테스트

In [26]:
model_checkpoint = "Tiabet/kogpt2-base-v2-finetuned-koGPT-complete_story"

model = GPT2LMHeadModel.from_pretrained(model_checkpoint)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_checkpoint, bos_token='<bos>', eos_token='<eos>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', sep_token = '<sep>', padding_side='left')

In [27]:
# Load the existing model checkpoint
model_config = GPT2Config.from_pretrained(model_checkpoint)

# Add the new special token to the model's config
model_config.bos_token_id = tokenizer.bos_token_id
model_config.eos_token_id = tokenizer.eos_token_id
model_config.unk_token_id = tokenizer.unk_token_id
model_config.pad_token_id = tokenizer.pad_token_id
model_config.mask_token_id = tokenizer.mask_token_id
model_config.sep_token_id = tokenizer.sep_token_id  # Add this line

# Initialize the model with the updated configuration
model = GPT2LMHeadModel.from_pretrained(model_checkpoint, config=model_config)

model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 51203. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(51203, 768)

In [28]:
import torch
from torch.utils.data import DataLoader


# Assuming you have a preprocessed test dataset with input_ids and attention_mask
test_input_ids = tokenized_datasets['test']['input_ids']  # List of input_ids for your test dataset
test_attention_mask = tokenized_datasets['test']['attention_mask']  # List of attention_mask for your test dataset

for i, (input_ids, attention_mask) in enumerate(zip(test_input_ids, test_attention_mask)):
    # Modify input_ids and attention_mask as needed
    modified_input_ids = input_ids[0][:-1]
    modified_attention_mask = attention_mask[0][:-1]

    # Save the modified values back to the test_input_ids and test_attention_mask lists
    test_input_ids[i] = modified_input_ids
    test_attention_mask[i] = modified_attention_mask



# Create a DataLoader to batch your test data
batch_size = 16  # You can adjust this according to your GPU's memory capacity
test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_input_ids), torch.tensor(test_attention_mask))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

generated_sentences = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Use torch.no_grad() to prevent unnecessary gradient computations
with torch.no_grad():
      for step,batch in enumerate(test_loader):
        if step%50 == 0:
          print(f"{step}번째 배치 시작")

        batch_input_ids, batch_attention_mask = [item.to(device) for item in batch]


        # Generate text using the model
        batch_output = model.generate(
            input_ids=batch_input_ids,
            do_sample=True,
            attention_mask=batch_attention_mask,
            max_length=128,
            top_k=5,
            top_p=0.9,
            temperature=0.1,
            repetition_penalty=2.0
        )

        # Decode and append the generated text for each example in the batch
        for output in batch_output:
            generated_text = tokenizer.decode(output, skip_special_tokens=True)
            generated_sentences.append(generated_text)


0번째 배치 시작
50번째 배치 시작
100번째 배치 시작
150번째 배치 시작
200번째 배치 시작
250번째 배치 시작
300번째 배치 시작
350번째 배치 시작
400번째 배치 시작
450번째 배치 시작
500번째 배치 시작
550번째 배치 시작
600번째 배치 시작
650번째 배치 시작
700번째 배치 시작
750번째 배치 시작
800번째 배치 시작
850번째 배치 시작
900번째 배치 시작


In [29]:
generated_sentences[0]

'첫 번째 문장 : 서영이는 기차에 타서 자리에 앉았다. 세 번째 문장 : 서영이는 바로 승무원에게 기차표를 보여줬다. 두 번째 문장 :나는 나는 그 지근데 그는서 있었다.'

In [30]:
max_sentence_length = 50  # Define your maximum desired sentence length

output_sentence = []

for input_text in generated_sentences:
    sentences = input_text.split(":")

    for i, sentence in enumerate(sentences):
        if "두 번째 문장" in sentence:
            if i + 1 < len(sentences):
                extracted_sentence = sentences[i + 1].strip()

                # Truncate the sentence if it exceeds the maximum length
                if len(extracted_sentence) > max_sentence_length:
                    extracted_sentence = extracted_sentence[:max_sentence_length]

                output_sentence.append(extracted_sentence)
                break


In [31]:
len(output_sentence)

15018

In [32]:
import json

def jsonlload(fname):
    with open(fname, "r", encoding="utf-8") as f:
        lines = f.read().strip().split("\n")
        j_list = [json.loads(line) for line in lines]

    return j_list


def jsonldump(j_list, fname):
    with open(fname, "w", encoding='utf-8') as f:
        for json_data in j_list:
            f.write(json.dumps(json_data, ensure_ascii=False)+'\n')


In [33]:
j_list = jsonlload("/content/drive/MyDrive/nikluge-sc-2023-test.jsonl")

In [34]:
for idx, oup in enumerate(output_sentence):
    j_list[idx]["output"] = oup

jsonldump(j_list, '/content/drive/MyDrive/kogpt_result_2.json')